# Reddit 댓글 규칙 위반 여부 분류

## 1. Import Library

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/jigsaw-agile-community-rules/sample_submission.csv
/kaggle/input/jigsaw-agile-community-rules/train.csv
/kaggle/input/jigsaw-agile-community-rules/test.csv


In [2]:
import pandas as pd
import numpy as np
import re

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.calibration import CalibratedClassifierCV

SEED = 42

## 2. Load Data

In [3]:
# 데이터 로드
train_df = pd.read_csv('/kaggle/input/jigsaw-agile-community-rules/train.csv')
test_df = pd.read_csv('/kaggle/input/jigsaw-agile-community-rules/test.csv')

# 데이터 크기 확인
print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")

Train shape: (2029, 9)
Test shape: (10, 8)


In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   row_id              2029 non-null   int64 
 1   body                2029 non-null   object
 2   rule                2029 non-null   object
 3   subreddit           2029 non-null   object
 4   positive_example_1  2029 non-null   object
 5   positive_example_2  2029 non-null   object
 6   negative_example_1  2029 non-null   object
 7   negative_example_2  2029 non-null   object
 8   rule_violation      2029 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 142.8+ KB


In [5]:
train_df.head()

,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1


In [6]:
# 클래스 불균형 확인
train_df["rule_violation"].value_counts()

rule_violation
1    1031
0     998
Name: count, dtype: int64

## 3. Data Preprocessing

In [7]:
def preprocess_text(text):
    # 소문자로 통일
    text = str(text).lower()

    # url이 포함된 경우 [URL] 토큰으로 치환
    text = re.sub(r'http\S+|www\S+', '[URL]', text)

    # 특수문자 제거
    text = re.sub(r'[^\w\s[\]]', '', text)

    # 공백 제거
    text = ' '.join(text.split())
    return text

In [8]:
body_processed = train_df["body"].apply(preprocess_text)
rule_processed = train_df["rule"].apply(preprocess_text)

# 타겟 컬럼 분리
X = body_processed + " [RULE] " + rule_processed
y = train_df["rule_violation"]

# 테스트 데이터에도 동일한 전처리 적용
body_processed_test = test_df["body"].apply(preprocess_text)
rule_processed_test = test_df["rule"].apply(preprocess_text)

X_test = body_processed_test + " [RULE] " + rule_processed_test

# 전처리된 샘플 데이터를 확인합니다.
print(f"Processed data sample: {X.iloc[0][:100]}")

Processed data sample: banks dont want you to know this click here to know more [RULE] no advertising spam referral links u


In [9]:
# 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [10]:
print(X)

0       banks dont want you to know this click here to...
1       sd stream [ eng link 1] [URL] [RULE] no advert...
2       lol try appealing the ban and say you wont do ...
3       she will come your home open her legs with and...
4       code free tyrande [imgur][URL] for you and you...
                              ...                        
2024    please edit your post so it is readable these ...
2025    yes and in a right to work state they can even...
2026    hd streams eng hd[ watch herepc mobile ][URL] ...
2027    no not when doing so obviously presents a safe...
2028    ca is an at fault state so they will not be ab...
Length: 2029, dtype: object


## 4. Create baseline models and Evaluation

In [11]:
# 분할했던 train, val 데이터 병합
X_all = X
y_all = y.to_numpy()

# TF-IDF 시행
tfidf = TfidfVectorizer(
    max_features=2000,
    min_df=2,
    max_df=0.8,
    ngram_range=(1,3),
    sublinear_tf=True,
    use_idf=True
)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)

pipe_lr = make_pipeline(
    tfidf,
    LogisticRegression(max_iter=2000, class_weight="balanced", random_state=SEED, solver="liblinear")
)

pipe_sgd = make_pipeline(
    tfidf,
    SGDClassifier(max_iter=2000, loss="log_loss", class_weight="balanced", random_state=SEED)
)

pipe_svc = make_pipeline(
    tfidf,
    CalibratedClassifierCV(LinearSVC(class_weight="balanced", random_state=SEED),
                          method="sigmoid", cv=3)
)

skf_models = {
    "Logistic Regression" : pipe_lr,
    "SGD" : pipe_sgd,
    "LinearSVC" : pipe_svc
}


oof_auc = {}
test_prob = {}

for name, pipe in skf_models.items():
    oof_proba = cross_val_predict(
        pipe, X_all, y_all, cv=skf, method="predict_proba", n_jobs=-1
    )[:, 1]
    oof_auc[name] = roc_auc_score(y_all, oof_proba)

    pipe.fit(X_all, y_all)
    test_prob[name] = pipe.predict_proba(X_test)[:, 1]


# 결과 확인
for k, v in oof_auc.items():
    print(f"{k:24s} OOF AUC: {v:.4f}")

Logistic Regression      OOF AUC: 0.7992
SGD                      OOF AUC: 0.8005
LinearSVC                OOF AUC: 0.7995


## 5. Submission

In [12]:
# 베이스라인 모델 중 성능이 높았던 SGD 선택
best_base = "SGD"
best_prob_base = test_prob[best_base]


# Ver 1. SGD 모델 제출
submission = pd.read_csv("/kaggle/input/jigsaw-agile-community-rules/sample_submission.csv", index_col=0)
single_submission = submission.copy()
single_submission["rule_violation"] = best_prob_base
single_submission.to_csv("single_submission.csv", index=False)

In [13]:
# 결과 확인
for name, arr in {
    "single": single_submission["rule_violation"].values
}.items():
    print(name, len(arr), np.isnan(arr).sum(), arr.min(), arr.max())

single 10 0 0.10552077002824435 0.8536586535811297


In [14]:
print("len(X_test) =", len(X_test))
print("len(sample_submission) =", len(submission))
print("len(single_submission) =", len(single_submission))

len(X_test) = 10
len(sample_submission) = 10
len(single_submission) = 10


In [15]:
final_sub = single_submission
final_sub.to_csv("/kaggle/working/submission.csv", index=False, float_format="%.6f")

In [16]:
MY_SUB_PATH = "/kaggle/working/submission.csv"
SAMPLE_PATH = "/kaggle/input/jigsaw-agile-community-rules/sample_submission.csv"

sample = pd.read_csv(SAMPLE_PATH)

def load_submission_anyway(path):
    try:
        sub = pd.read_csv(path, encoding="utf-8-sig")
    except Exception:
        sub = None

    if sub is None or ("row_id" not in sub.columns or "rule_violation" not in sub.columns):
        try:
            sub2 = pd.read_csv(path, header=None, encoding="utf-8-sig")
        except Exception:
            sub2 = None
        if sub2 is not None:
            if sub2.shape[1] == 1:
                sub2 = sub2[0].str.split(",", expand=True)
            if sub2.shape[0] >= 1 and any(x in str(sub2.iloc[0,0]).lower() for x in ["row", "id"]):
                sub2.columns = sub2.iloc[0]
                sub2 = sub2.iloc[1:].reset_index(drop=True)
            if sub2.shape[1] >= 2:
                sub2 = sub2.iloc[:, :2]
                sub2.columns = ["row_id", "rule_violation"]
            sub = sub2

    if "row_id" not in sub.columns:
        first = sub.columns[0]
        try:
            sub[first] = pd.to_numeric(sub[first], errors="coerce").astype("Int64")
            sub.rename(columns={first:"row_id"}, inplace=True)
        except Exception:
            tmp = sample.copy()
            cand_cols = [c for c in sub.columns if c != "row_id"]
            rv = pd.to_numeric(sub[cand_cols[-1]], errors="coerce") if cand_cols else pd.Series(np.full(len(tmp), 0.5))
            tmp["rule_violation"] = rv.values[:len(tmp)]
            return tmp

    sub.columns = [str(c).strip().lstrip("\ufeff") for c in sub.columns]
    keep = ["row_id", "rule_violation"]
    sub = sub[[c for c in sub.columns if c in keep]]
    for c in keep:
        if c not in sub.columns:
            sub[c] = np.nan
    sub = sub[keep]
    return sub

sub = load_submission_anyway(MY_SUB_PATH)

sub["row_id"] = pd.to_numeric(sub["row_id"], errors="coerce").astype("Int64")
rv = pd.to_numeric(sub["rule_violation"], errors="coerce")
rv[~np.isfinite(rv)] = 0.5
sub["rule_violation"] = rv.clip(0,1).astype(float)

sub = sample[["row_id"]].merge(sub, on="row_id", how="left")
if sub["rule_violation"].isna().any():
    sub["rule_violation"].fillna(0.5, inplace=True)

# 최종 저장
sub.to_csv("/kaggle/working/submission.csv", index=False, float_format="%.6f")

print("FINAL:", sub.shape, list(sub.columns), sub.isna().sum().to_dict(),
      sub["rule_violation"].min(), sub["rule_violation"].max())

FINAL: (10, 2) ['row_id', 'rule_violation'] {'row_id': 0, 'rule_violation': 0} 0.105521 0.853659
